In [1]:
import numpy as np
import osmnx as ox
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import random
import pickle
import pymde
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding, SpectralEmbedding
from scipy import sparse

import mlrfit as mf
import lrrouting as ldr

import cvxpy as cp
import numba as nb

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

#  Matrix definition

In [3]:
rank = 6

mtype = "half_moon" #"stanford"
place = "Half Moon Bay, CA, USA"#"Stanford, CA, USA"
G, Adj, Dist, nodes_cc = ldr.dist_matrix_osmnx(place, directed=True, nodes=True)

A = Dist
m = n = A.shape[0]
diam_G = A.max()
w_min = A[A>0].min()
print(m, diam_G, w_min, (Adj>0).sum())

[345]
n_cc = 345, n0 = 345
in  degrees: {0: 0, 1: 110, 2: 36, 3: 134, 4: 65}
out degrees: {0: 0, 1: 108, 2: 40, 3: 132, 4: 65}
345 45.0 1.0 844


In [4]:
assert nx.is_strongly_connected(G)
np.histogram(Dist.flatten(), bins=5, density=True)

(array([0.0223025 , 0.04925763, 0.03059955, 0.00848468, 0.00046676]),
 array([ 0.,  9., 18., 27., 36., 45.]))

In [5]:
adjacency_list = ldr.adjacency_directed_list(Adj)
sources, targets = ldr.st_pairs(n, Dist, 1020)
M = min(1000, sources.size)
sources = sources[:M]
targets = targets[:M]

In [6]:
PSD = False
w_min = A[A>0].min()
rt_max_iters = min(int(5*A.max()/w_min), (10**4) // 2)
symm = np.allclose(A, A.T)
print(f"{symm=}")
filename = "%s_r%d_%d"%(mtype, rank, n)

symm=False


In [7]:
np.histogram(Adj[Adj>0], bins=5, density=True)

(array([4.96445498, 0.        , 0.        , 0.        , 0.03554502]),
 array([1. , 1.2, 1.4, 1.6, 1.8, 2. ]))

In [8]:
info = {} 

In [9]:
fraction_of_nodes = 0.1
pi_rows = np.random.permutation(n)[:int(n * fraction_of_nodes)]
pi_cols = pi_rows #np.random.permutation(n)[:int(n * fraction_of_nodes)]

pi_row_Dist = ldr.sparse_sampled_matrix(pi_rows, Dist, rows=True)
pi_col_Dist_T = ldr.sparse_sampled_matrix(pi_cols, Dist, rows=False)

rDist = np.zeros((n, n))
rDist[pi_rows] = Dist[pi_rows]
assert np.allclose(pi_row_Dist.toarray(), rDist)

cDist = np.zeros((n, n))
cDist[:, pi_cols] = Dist[:, pi_cols]
assert np.allclose(pi_col_Dist_T.T.toarray(), cDist)
print("PASSED")

rDist = Dist[pi_rows]
cDist = Dist[:, pi_cols].T

pi_rows_c = np.delete(np.arange(n), pi_rows)
pi_cols_c = np.delete(np.arange(n), pi_cols)

PASSED


In [10]:
for rank in [4, 8, 16, 20]:
    print("\n\nSymmetric fit -> split -> asymmetric fit")
    # get symmetric embedding
    Z_symm, loss, losses = ldr.fast_cc(rank//2, pi_rows=pi_rows, pi_rows_c=pi_rows_c, rDist=(rDist+cDist)/2, symm=True, n_init=5,
                                        max_iter=1000, eps=1e-6, verbose=False, freq=500)
    print(f"\n{rank/2=}, {losses[0]=}, {losses[-1]=}")
    # split and fit asymmetric embedding
    l_dar = ldr.construct_node_embedding_graph(Z_symm, adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}
    Z0 = np.concatenate([Z_symm, Z_symm], axis=0)
    Z, loss, losses = ldr.fast_cc(rank, pi_rows, pi_cols, pi_rows_c, pi_cols_c, rDist, cDist, Z0=Z0, n_init=2,
                                    max_iter=1000, eps=1e-6, verbose=False, freq=200)
    print(f"\n{rank=}, {losses[0]=}, {losses[-1]=}")
    print(r"$\|x_i-y_j\|_2$")
    l_dar = ldr.construct_xy_node_embedding_graph(Z[:n], Z[n:], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}


    print("\n\nAsymmetric fit")
    Z, loss, losses = ldr.fast_cc(rank, pi_rows, pi_cols, pi_rows_c, pi_cols_c, rDist, cDist, n_init=2, 
                                  max_iter=1000, eps=1e-6, verbose=False, freq=500)
    print(f"\n{rank=}, {losses[0]=}, {losses[-1]=}")
    print(r"$\|x_i-y_j\|_2$")
    l_dar = ldr.construct_xy_node_embedding_graph(Z[:n], Z[n:], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}
    print(r"$\|x_i-x_j\|_2$")
    l_dar = ldr.construct_node_embedding_graph(Z[:n], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}
    print(r"$\|y_i-y_j\|_2$")
    l_dar = ldr.construct_node_embedding_graph(Z[n:], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}



Symmetric fit -> split -> asymmetric fit


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/parshakova.tanya/Documents/projects/lr_distributed_routing/lrrouting/cg.py:48: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  rho0 = np.dot(r, z)
/Users/parshakova.tanya/Documents/projects/lr_distributed_routing/lrrouting/cg.py:72: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  losses[k] = np.sqrt(np.dot(r, r)) / b_norm



rank/2=2.0, losses[0]=467.18218703841916, losses[-1]=0.0
median_stretch=120.0%, mean_stretch=191.3%
%[ratio<2] = 78.50%, %[ratio<1.2] = 50.10%, %[ratio=1.] = 36.80%

rank=4, losses[0]=11.817793557438698, losses[-1]=0.0
$\|x_i-y_j\|_2$
median_stretch=163.6%, mean_stretch=234.1%
%[ratio<2] = 61.80%, %[ratio<1.2] = 31.70%, %[ratio=1.] = 22.90%


Asymmetric fit

rank=4, losses[0]=445.2723753172062, losses[-1]=0.0
$\|x_i-y_j\|_2$
median_stretch=162.5%, mean_stretch=258.5%
%[ratio<2] = 60.60%, %[ratio<1.2] = 30.30%, %[ratio=1.] = 17.50%
$\|x_i-x_j\|_2$
median_stretch=160.0%, mean_stretch=215.8%
%[ratio<2] = 67.10%, %[ratio<1.2] = 23.00%, %[ratio=1.] = 16.10%
$\|y_i-y_j\|_2$
median_stretch=192.2%, mean_stretch=246.1%
%[ratio<2] = 54.50%, %[ratio<1.2] = 14.90%, %[ratio=1.] = 9.80%


Symmetric fit -> split -> asymmetric fit

rank/2=4.0, losses[0]=423.3294465846347, losses[-1]=2.129159674165943
median_stretch=100.0%, mean_stretch=121.4%
%[ratio<2] = 95.60%, %[ratio<1.2] = 76.20%, %[ratio=1.] = 

In [11]:
mf.rel_diff(Dist, Dist.T)

0.1238628157246641

In [12]:
for rank in [4, 8, 16, 20]:
    print("\n\nSymmetric fit -> split -> asymmetric fit")
    # get symmetric embedding
    Z_symm, loss, losses = ldr.fast_cc(rank//2, pi_rows=pi_rows, pi_rows_c=pi_rows_c, rDist=rDist, symm=True, n_init=5,
                                        max_iter=1000, eps=1e-6, verbose=False, freq=500)
    print(f"\n{rank/2=}, {losses[0]=}, {losses[-1]=}")
    # split and fit asymmetric embedding
    l_dar = ldr.construct_node_embedding_graph(Z_symm, adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}
    Z0 = np.concatenate([Z_symm, Z_symm], axis=0)
    Z, loss, losses = ldr.fast_cc(rank, pi_rows, pi_cols, pi_rows_c, pi_cols_c, rDist, cDist, Z0=Z0, n_init=2,
                                    max_iter=1000, eps=1e-6, verbose=False, freq=200)
    print(f"\n{rank=}, {losses[0]=}, {losses[-1]=}")
    print(r"$\|x_i-y_j\|_2$")
    l_dar = ldr.construct_xy_node_embedding_graph(Z[:n], Z[n:], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}


    print("\n\nAsymmetric fit")
    Z, loss, losses = ldr.fast_cc(rank, pi_rows, pi_cols, pi_rows_c, pi_cols_c, rDist, cDist, n_init=2, 
                                  max_iter=1000, eps=1e-6, verbose=False, freq=500)
    print(f"\n{rank=}, {losses[0]=}, {losses[-1]=}")
    print(r"$\|x_i-y_j\|_2$")
    l_dar = ldr.construct_xy_node_embedding_graph(Z[:n], Z[n:], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}
    print(r"$\|x_i-x_j\|_2$")
    l_dar = ldr.construct_node_embedding_graph(Z[:n], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}
    print(r"$\|y_i-y_j\|_2$")
    l_dar = ldr.construct_node_embedding_graph(Z[n:], adjacency_list)
    _ = {'ratios' : ldr.subopt_ratios(l_dar, Dist, sources, targets)}



Symmetric fit -> split -> asymmetric fit

rank/2=2.0, losses[0]=462.2327196719228, losses[-1]=0.0
median_stretch=157.1%, mean_stretch=237.0%
%[ratio<2] = 62.80%, %[ratio<1.2] = 24.70%, %[ratio=1.] = 12.90%

rank=4, losses[0]=12.871487702449661, losses[-1]=0.0
$\|x_i-y_j\|_2$
median_stretch=147.2%, mean_stretch=229.5%
%[ratio<2] = 65.20%, %[ratio<1.2] = 36.30%, %[ratio=1.] = 22.30%


Asymmetric fit

rank=4, losses[0]=440.12489830985993, losses[-1]=0.0
$\|x_i-y_j\|_2$
median_stretch=127.3%, mean_stretch=219.9%
%[ratio<2] = 67.10%, %[ratio<1.2] = 45.10%, %[ratio=1.] = 32.30%
$\|x_i-x_j\|_2$
median_stretch=120.9%, mean_stretch=195.1%
%[ratio<2] = 74.50%, %[ratio<1.2] = 49.90%, %[ratio=1.] = 39.30%
$\|y_i-y_j\|_2$
median_stretch=180.0%, mean_stretch=245.4%
%[ratio<2] = 59.80%, %[ratio<1.2] = 23.40%, %[ratio=1.] = 16.40%


Symmetric fit -> split -> asymmetric fit

rank/2=4.0, losses[0]=416.9441193563446, losses[-1]=2.410265406946407
median_stretch=114.6%, mean_stretch=142.6%
%[ratio<2] = 8